In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.compose import make_column_transformer
from matplotlib import pyplot as plt
from scipy.stats import zscore

In [2]:
df = pd.read_csv('Property_structured_data.csv')

In [3]:
initial_df = df.shape

In [4]:
initial_df_columns = df.columns.to_list()

## Clean Database

### Price (float)

In [5]:
df.loc[df["price"] >= 800000, "type_of_property"] = "OTHER"

In [6]:
df= df.loc[df['price'] != -1]
df.shape

(59429, 23)

### Surface (float)

In [7]:
df = df.loc[df['surface'] != -1]
df.shape

(49412, 23)

### Sub Type of property (filter and drop)

In [8]:
df['subtype_of_property'].value_counts()

HOUSE                   20679
APARTMENT               15864
VILLA                    2954
APARTMENT_BLOCK          1414
DUPLEX                   1247
GROUND_FLOOR             1146
PENTHOUSE                1009
MIXED_USE_BUILDING        993
FLAT_STUDIO               693
EXCEPTIONAL_PROPERTY      636
MANSION                   494
SERVICE_FLAT              483
TOWN_HOUSE                376
COUNTRY_COTTAGE           301
BUNGALOW                  243
LOFT                      231
FARMHOUSE                 155
TRIPLEX                   119
CHALET                    107
MANOR_HOUSE                76
OTHER_PROPERTY             72
CASTLE                     61
KOT                        58
PAVILION                    1
Name: subtype_of_property, dtype: int64

In [9]:
df= df.loc[df['subtype_of_property'] != 'APARTMENT_BLOCK']
df.shape

(47998, 23)

In [10]:
df= df.loc[df['subtype_of_property'] != 'MIXED_USE_BUILDING']
df.shape

(47005, 23)

### Number of bedrooms (int)

In [11]:
df = df.loc[df['number_of_bedrooms'] <200]

In [12]:
df['number_of_bedrooms'] = df['number_of_bedrooms'].replace(-1,0)

In [13]:
df['number_of_bedrooms'].value_counts()

3     15278
2     14724
4      6945
1      4875
5      2662
6      1032
0       755
7       323
8       168
9        89
10       52
11       21
12       21
14       13
13        7
16        5
15        5
20        4
17        4
18        3
25        3
19        2
28        2
22        2
23        1
26        1
66        1
70        1
24        1
45        1
30        1
31        1
Name: number_of_bedrooms, dtype: int64

### Zip Code (category)

#### XX

In [14]:
df['zip_code_xx'] = df['postal_code'].apply(lambda x : 'be_zip_'+str(x)[:2])

In [15]:
df['zip_code_xx'].value_counts()

be_zip_10    3496
be_zip_11    1661
be_zip_83    1628
be_zip_20    1336
be_zip_84    1275
             ... 
be_zip_65     161
be_zip_79     131
be_zip_76     111
be_zip_47      88
be_zip_64      53
Name: zip_code_xx, Length: 80, dtype: int64

#### X

In [16]:
df['zip_code_x'] = df['postal_code'].apply(lambda x : 'be_zip_'+str(x)[:1])

In [17]:
df['zip_code_x'].value_counts()

be_zip_1    10036
be_zip_2     7795
be_zip_8     7080
be_zip_9     6314
be_zip_3     4096
be_zip_4     3742
be_zip_7     3111
be_zip_6     3077
be_zip_5     1752
Name: zip_code_x, dtype: int64

### Land surface (float)

In [18]:
#df.loc[(df['land_surface'] == -1) & (df['garden_surface'] > 0 )]

In [19]:
df['land_surface'] = [land_surface if land_surface != -1 else garden_surface if garden_surface > 0 else land_surface for garden_surface,land_surface in zip(df['garden_surface'],df['land_surface'])]

In [20]:
df['land_surface'] = df['land_surface'].replace(-1,0)

In [21]:
df['land_surface'].value_counts()

0         21668
100         209
200         193
300         163
150         157
          ...  
3584          1
3051          1
7069          1
8030          1
175000        1
Name: land_surface, Length: 3528, dtype: int64

### Garden (0,1)

In [22]:
df['garden'] = df['garden'].replace(-1,0)

In [23]:
df['garden'].value_counts()

0    30839
1    16164
Name: garden, dtype: int64

### Garden surface (float)

In [24]:
df['garden_surface'] = df['garden_surface'].replace(-1,0)
df['garden_surface'] = df['garden_surface'].replace(1,0)

In [25]:
df['garden_surface'].value_counts()

0       37109
100       304
200       270
50        198
300       196
        ...  
292         1
1115        1
568         1
1043        1
7200        1
Name: garden_surface, Length: 1413, dtype: int64

### Fully equiped kitchen  (0,1)

In [26]:
#df["fully_equipped_kitchen"] = df["fully_equipped_kitchen"].map({"-1.0": 0.25, "1.0": 1,"-1": 0.25, "1": 1, "INSTALLED": 0.5, "SEMI_EQUIPPED": 0.75, "NOT_INSTALLED": 0, "USA_INSTALLED": 0.5, "USA_SEMI_EQUIPPED": 0.75, "USA_UNINSTALLED": 0})

In [27]:
df["fully_equipped_kitchen"] = df["fully_equipped_kitchen"].map({"-1.0": 0, "1.0": 1,"-1": 0, "1": 1, "INSTALLED": 0, "SEMI_EQUIPPED": 1, "NOT_INSTALLED": 0, "USA_INSTALLED": 0, "USA_SEMI_EQUIPPED": 1, "USA_UNINSTALLED": 0})

In [28]:
df["fully_equipped_kitchen"].value_counts()

0    34280
1    12723
Name: fully_equipped_kitchen, dtype: int64

### Swiming pool  (0,1)

In [29]:
df['swimming_pool'] = df['swimming_pool'].replace(-1,0)

In [30]:
df['swimming_pool'].value_counts()

0    45863
1     1140
Name: swimming_pool, dtype: int64

### Furnished (0,1)

In [31]:
df['furnished'] = df['furnished'].replace(-1,0)

In [32]:
df['furnished'].value_counts()

0    45754
1     1249
Name: furnished, dtype: int64

### Open fire (0,1)

In [33]:
df['open_fire'] = df['open_fire'].replace(-1,0)

In [34]:
df['open_fire'].value_counts()

0    44777
1     2226
Name: open_fire, dtype: int64

### Terrace (0,1)

In [35]:
df['terrace'] = df['terrace'].replace(-1,0)

In [36]:
df['terrace'].value_counts()

1    30074
0    16929
Name: terrace, dtype: int64

### Terrace surface (float)

we have two terraces > 1000 m2 !

In [37]:
df = df.loc[df['terrace_surface'] < 500]

In [38]:
df['terrace_surface'] = df['terrace_surface'].replace(-1,0)

In [39]:
df['terrace_surface'].value_counts()

0      27369
20      1197
10      1088
15       948
12       870
       ...  
137        1
190        1
210        1
370        1
290        1
Name: terrace_surface, Length: 195, dtype: int64

### Facades (int)

In [40]:
df = df.loc[df["number_of_facades"] < 9]

In [41]:
df["number_of_facades"] = np.where((df["number_of_facades"] == -1) & (df["type_of_property"] == "APARTMENT"), 1, df["number_of_facades"])
df["number_of_facades"] = np.where((df["number_of_facades"] == -1) & (df["type_of_property"] == "HOUSE"), 2, df["number_of_facades"])

In [42]:
df = df.loc[df["number_of_facades"] != -1]

In [43]:
df['number_of_facades'].value_counts()

2    20461
4     9802
3     8238
1     7783
6        6
5        6
8        4
Name: number_of_facades, dtype: int64

### State of the building (category)

Tranform this part to category

In [44]:
df["state_of_the_building"] = df["state_of_the_building"].map({
    "NO_INFO": "TO_RENOVATE",#0.25, 
    "TO_BE_DONE_UP": "JUST_RENOVATED",#0.75, 
    "TO_RENOVATE": "TO_RENOVATE",#0.25, 
    "TO_RESTORE": "TO_REBUILD",#0.25, 
    "JUST_RENOVATED": "JUST_RENOVATED",#0.75, 
    "GOOD": "GOOD",#0.5, 
    "AS_NEW": "NEW"#1.0
})

In [45]:
df["state_of_the_building"].value_counts()

TO_RENOVATE       16329
GOOD              12701
NEW               10961
JUST_RENOVATED     6123
TO_REBUILD          186
Name: state_of_the_building, dtype: int64

### Type of property (category)

Separate into HOUSE, APARTMENTS and OTHERS

In [46]:
others = ["CHALET", "MANOR_HOUSE", "OTHER_PROPERTY", "CASTLE", "PAVILION"]

In [47]:
df.loc[df["price"] >= 800000, "type_of_property"] = "OTHER"

In [48]:
df.loc[df["subtype_of_property"].isin(others), "type_of_property"] = "OTHER"

In [49]:
df['type_of_property'].value_counts()

HOUSE        23583
APARTMENT    19819
OTHER         2898
Name: type_of_property, dtype: int64

### Price/m2

In [50]:
df['price_m2'] = df['price']/df['surface']

In [51]:
df['price_m2'].value_counts()

2500.000000    274
3000.000000    173
2000.000000    169
1500.000000    108
2750.000000    107
              ... 
4299.065421      1
3307.228916      1
3312.169014      1
1605.539474      1
1705.038760      1
Name: price_m2, Length: 20104, dtype: int64

### API list

In [52]:
api_list = ['surface','type_of_property','number_of_bedrooms','postal_code','land_surface','garden','garden_surface','fully_equipped_kitchen','swimming_pool','furnished','open_fire','terrace','terrace_surface','number_of_facades','state_of_the_building','zip_code_xx','zip_code_x','price']

In [53]:
df = df[api_list]

In [54]:
claned_df = df.shape

## Check database

In [55]:
print(f'initial_df {initial_df} claned_df {claned_df}')

initial_df (62430, 23) claned_df (46300, 18)


In [56]:
df.describe()

,surface,number_of_bedrooms,postal_code,land_surface,garden,garden_surface,fully_equipped_kitchen,swimming_pool,furnished,open_fire,terrace,terrace_surface,number_of_facades,price
count,46300.000000,46300.000000,46300.000000,4.630000e+04,46300.000000,46300.000000,46300.000000,46300.000000,46300.000000,46300.000000,46300.000000,46300.000000,46300.000000,4.630000e+04
mean,165.168812,2.833089,4984.067948,8.325338e+02,0.343024,182.958488,0.270302,0.023153,0.026220,0.046739,0.639417,10.281533,2.434665,3.962921e+05
std,207.372201,1.466459,3040.887123,9.160139e+03,0.474724,2394.490809,0.444121,0.150392,0.159792,0.211081,0.480175,21.609825,1.005131,3.977620e+05
min,1.000000,0.000000,1000.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.500000e+03
25%,94.000000,2.000000,2140.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.249000e+05
50%,134.000000,3.000000,4219.000000,7.600000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,2.990000e+05
75%,191.000000,3.000000,8370.000000,4.730000e+02,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,14.000000,3.000000,4.300000e+05
max,22617.000000,66.000000,9992.000000,1.100000e+06,1.000000,150000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,420.000000,8.000000,9.999999e+06


In [57]:
for index,atribute in enumerate(df.columns.to_list()):
    print(atribute)
    print(df[df.columns.to_list()[index]].value_counts())
    print('\n')

surface
100    887
150    880
90     833
120    804
110    714
      ... 
678      1
707      1
708      1
487      1
433      1
Name: surface, Length: 813, dtype: int64


type_of_property
HOUSE        23583
APARTMENT    19819
OTHER         2898
Name: type_of_property, dtype: int64


number_of_bedrooms
3     15023
2     14606
4      6822
1      4869
5      2585
6       976
0       747
7       299
8       157
9        83
10       45
12       18
11       18
14       11
13        7
16        5
15        5
17        4
18        3
20        3
25        3
22        2
23        1
45        1
24        1
66        1
28        1
26        1
19        1
30        1
31        1
Name: number_of_bedrooms, dtype: int64


postal_code
1180    707
1000    678
9000    663
8300    585
8400    569
       ... 
6781      1
6542      1
6922      1
6972      1
6674      1
Name: postal_code, Length: 1048, dtype: int64


land_surface
0         21317
100         208
200         192
300         163
150         15

# Data preprocessing

In [58]:
transformer = make_column_transformer(
    (OneHotEncoder(), ['type_of_property', 'zip_code_x','state_of_the_building']),
    remainder='passthrough')

In [59]:
transformed = transformer.fit_transform(df)
transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names())

/home/david/MyGithub/challenge-regression/env/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [60]:
transformed_df.columns.to_list()

['onehotencoder__x0_APARTMENT',
 'onehotencoder__x0_HOUSE',
 'onehotencoder__x0_OTHER',
 'onehotencoder__x1_be_zip_1',
 'onehotencoder__x1_be_zip_2',
 'onehotencoder__x1_be_zip_3',
 'onehotencoder__x1_be_zip_4',
 'onehotencoder__x1_be_zip_5',
 'onehotencoder__x1_be_zip_6',
 'onehotencoder__x1_be_zip_7',
 'onehotencoder__x1_be_zip_8',
 'onehotencoder__x1_be_zip_9',
 'onehotencoder__x2_GOOD',
 'onehotencoder__x2_JUST_RENOVATED',
 'onehotencoder__x2_NEW',
 'onehotencoder__x2_TO_REBUILD',
 'onehotencoder__x2_TO_RENOVATE',
 'surface',
 'number_of_bedrooms',
 'postal_code',
 'land_surface',
 'garden',
 'garden_surface',
 'fully_equipped_kitchen',
 'swimming_pool',
 'furnished',
 'open_fire',
 'terrace',
 'terrace_surface',
 'number_of_facades',
 'zip_code_xx',
 'price']

### Drop values from onehotencoder

In [61]:
drop_list = ['onehotencoder__x0_APARTMENT',
 'onehotencoder__x0_HOUSE',
 'onehotencoder__x0_OTHER',
 'onehotencoder__x1_be_zip_1',
 'onehotencoder__x1_be_zip_2',
 'onehotencoder__x1_be_zip_3',
 'onehotencoder__x1_be_zip_4',
 'onehotencoder__x1_be_zip_5',
 'onehotencoder__x1_be_zip_6',
 'onehotencoder__x1_be_zip_7',
 'onehotencoder__x1_be_zip_8',
 #'onehotencoder__x1_be_zip_9',
 'onehotencoder__x2_GOOD',
 'onehotencoder__x2_JUST_RENOVATED',
 'onehotencoder__x2_NEW',
 'onehotencoder__x2_TO_REBUILD',
 #'onehotencoder__x2_TO_RENOVATE',
 'surface',
 'number_of_bedrooms',
 #'postal_code',
 'land_surface',
 'garden',
 'garden_surface',
 'fully_equipped_kitchen',
 'swimming_pool',
 'furnished',
 'open_fire',
 'terrace',
 'terrace_surface',
 'number_of_facades',
 #'zip_code_xx',
 'price']

In [62]:
transformed_df = transformed_df[drop_list]

In [70]:
for index,atribute in enumerate(transformed_df.columns.to_list()):
    print(atribute)
    print(transformed_df[transformed_df.columns.to_list()[index]].value_counts())
    print('\n')

onehotencoder__x0_APARTMENT
0.0    26481
1.0    19819
Name: onehotencoder__x0_APARTMENT, dtype: int64


onehotencoder__x0_HOUSE
1.0    23583
0.0    22717
Name: onehotencoder__x0_HOUSE, dtype: int64


onehotencoder__x0_OTHER
0.0    43402
1.0     2898
Name: onehotencoder__x0_OTHER, dtype: int64


onehotencoder__x1_be_zip_1
0.0    36413
1.0     9887
Name: onehotencoder__x1_be_zip_1, dtype: int64


onehotencoder__x1_be_zip_2
0.0    38633
1.0     7667
Name: onehotencoder__x1_be_zip_2, dtype: int64


onehotencoder__x1_be_zip_3
0.0    42265
1.0     4035
Name: onehotencoder__x1_be_zip_3, dtype: int64


onehotencoder__x1_be_zip_4
0.0    42571
1.0     3729
Name: onehotencoder__x1_be_zip_4, dtype: int64


onehotencoder__x1_be_zip_5
0.0    44552
1.0     1748
Name: onehotencoder__x1_be_zip_5, dtype: int64


onehotencoder__x1_be_zip_6
0.0    43233
1.0     3067
Name: onehotencoder__x1_be_zip_6, dtype: int64


onehotencoder__x1_be_zip_7
0.0    43200
1.0     3100
Name: onehotencoder__x1_be_zip_7, dtype

# Split Data

In [63]:
y = transformed_df.iloc[:,-1].values  #price
x = transformed_df.iloc[:,:-1].values  #rest

In [64]:
x.shape

(46300, 27)

In [65]:
x

array([[0.0, 1.0, 0.0, ..., 1, 0, 3],
       [1.0, 0.0, 0.0, ..., 1, 0, 3],
       [1.0, 0.0, 0.0, ..., 1, 6, 2],
       ...,
       [0.0, 1.0, 0.0, ..., 1, 30, 2],
       [1.0, 0.0, 0.0, ..., 1, 0, 1],
       [0.0, 1.0, 0.0, ..., 1, 40, 4]], dtype=object)

In [66]:
y.shape

(46300,)

In [67]:
y

array([100000, 219000, 285000, ..., 235000, 316200, 210000], dtype=object)